In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import json
from itertools import chain
from rich import print
import sys
import os
import utils

# Đường dẫn đến thư mục mini_model
mini_model_path = os.path.abspath(os.path.join(os.getcwd(), ''))

# Lấy danh sách tất cả thư mục con trong mini_model
for folder in os.listdir(mini_model_path):
    folder_path = os.path.join(mini_model_path, folder)
    # Chỉ thêm nếu là thư mục và không phải file
    if os.path.isdir(folder_path):
        if folder_path not in sys.path:
            sys.path.append(folder_path)

import function_get_score_amenities as fga
import function_get_score_rate as fgr
import CF_user_review.CF_user_scores as cfus
import recommenr_review_data as rhr
import review_quality
import recommend_location as rl
import recommend_polices as rp
import recommed_room as rr
import function_get_score_price as fgsp


In [2]:
from pathlib import Path
import config
DEFAULT_BASE_DIR = Path("D:/graduate_dissertation/final")

In [3]:
with open(config.get_path("feature_sub_rate.json"), 'r', encoding='utf-8') as f:
    feature_sub_rate = json.load(f)

with open(config.get_path("weights_criteria_utilities.json"), 'r', encoding='utf-8') as f:
    weights_criteria_utilities = json.load(f)

with open(config.get_path("feature_popular_facilities.json"), 'r', encoding='utf-8') as f:
    feature_popular_facilities = json.load(f)
    
with open(config.get_path("feature_facilities.json"), 'r', encoding='utf-8') as f:
    feature_facilities = json.load(f)

with open(config.get_path("feature_url.json"), 'r', encoding='utf-8') as f:
    feature_url = json.load(f)

with open(config.get_path("feature_allRoom.json"), 'r', encoding='utf-8') as f:
    feature_allRoom = json.load(f)
    
with open(config.get_path("review_processing.json"), 'r', encoding='utf-8') as f:
    result_groupby_id= json.load(f)

with open(config.get_path("feature_location.json"), 'r', encoding='utf-8') as f:
    feature_location = json.load(f)

with open(config.get_path("feature_policies.json"), 'r', encoding='utf-8') as f:
    feature_policies = json.load(f)

with open(config.get_path('hotel_data_room.json'), 'r', encoding='utf-8') as f:
    feature_detail_room = json.load(f)

with open(config.get_path('feature_star_rating.json'), 'r', encoding='utf-8') as f:
    feature_star_rating = json.load(f)


#### Load model

In [4]:
recommend_faclities_hotel = fga.HotelSimilarityRecommender(    
    model_name='paraphrase-multilingual-MiniLM-L12-v2',
    use_gpu=True,
    model_dir=DEFAULT_BASE_DIR.joinpath("mini_model\\function_get_score_amenities\\model_similarity_amenities\\hotel"),
    type='hotel',
    batch_size=256,
    faiss_metric='IP'
)
recommend_faclities_hotel.load_model()
recommend_faclities_room = fga.HotelSimilarityRecommender(
    model_name='paraphrase-multilingual-MiniLM-L12-v2',
    use_gpu=True,
    model_dir=DEFAULT_BASE_DIR.joinpath("mini_model\\function_get_score_amenities\\model_similarity_amenities\\room"),
    type='room',
    batch_size=256,
    faiss_metric='IP'
)
recommend_faclities_room.load_model()


Initializing Sentence-BERT model: paraphrase-multilingual-MiniLM-L12-v2
Using CPU for Sentence-BERT model: paraphrase-multilingual-MiniLM-L12-v2
SafeTensors file D:\graduate_dissertation\final\mini_model\function_get_score_amenities\model_similarity_amenities\hotel\model.safetensors not found, initializing new model
Initialized new model paraphrase-multilingual-MiniLM-L12-v2 on CPU
Loaded hotel data from D:\graduate_dissertation\final\mini_model\function_get_score_amenities\model_similarity_amenities\hotel\hotel_data.pkl
Loaded vectors from D:\graduate_dissertation\final\mini_model\function_get_score_amenities\model_similarity_amenities\hotel\hotel_vectors.npy
Loaded Faiss index from D:\graduate_dissertation\final\mini_model\function_get_score_amenities\model_similarity_amenities\hotel\faiss_index.bin
Initializing Sentence-BERT model: paraphrase-multilingual-MiniLM-L12-v2
Using CPU for Sentence-BERT model: paraphrase-multilingual-MiniLM-L12-v2
SafeTensors file D:\graduate_dissertation\

In [5]:
CF = cfus()
state_model = CF.load_model(DEFAULT_BASE_DIR.joinpath('mini_model\\CF_user_review\\CF_state_for_user.pkl'))
country_model = CF.load_model(DEFAULT_BASE_DIR.joinpath('mini_model\\CF_user_review\\CF_country_for_user.pkl'))

In [ ]:
hotel_review_stats = review_quality.HotelReviewStats()
hotel_review_stats.load_stats()

In [4]:
query = {
    # ====== location ======
    'province': 'Thành phố Hồ Chí Minh',
    'nearby_places': ["Cố đô Hoa Lư", "Vườn Chim Thung Nham", "Ninh Binh Train Station"],
    'is_near_center': True,
    'public_transport_access': ['ga tàu'], # đã xử lý 50%, 
    # ======
    'price_range': (300000, 1000000),  
    'rating': (3, 5),
    'stars_rating': 3,
    'services': ["Dịch vụ sự kiện & hỗ trợ đặc biệt"], 
    'criteria': ['Vị trí & môi trường'],
    'amenities': ["TV màn hình phẳng", "két an toàn"],
    'room_type': ['phòng đơn', 'phòng đôi'], # tạm chưa xử lý
    'booking_flexibility': ['hủy miễn phí', 'thanh toán khi nhận phòng'], # tạm chưa xử lý
    'distance_to_city_center': 5,
    
    'country': 'Việt Nam',
    'state': 'Cặp đôi',

    # ====== policies ======
    'policies': {
        "Nhận phòng": '11:00-12:00',
        "Trả phòng": '12:00-13:00',
        "Chỉ thanh toán bằng tiền mặt": None,
        "Các phương thức thanh toán được chấp nhận": 'Bankcard',
        "Giới hạn độ tuổi": None,
        "Giờ giới nghiêm": 'Cổng vào chỗ nghỉ sẽ đóng trong khoảng 00:00-5:00',
        "Hút thuốc": 'Không cho phép hút thuốc.',
        "Hủy đặt phòng/ Trả trước": 'Các chính sách hủy và thanh toán trước',
        "Không giới hạn độ tuổi": 'Không có yêu cầu về độ tuổi',
        "Nhóm": None,
        "Thẻ được chấp nhận tại chỗ nghỉ này": None,
        "Thẻ được chấp nhận tại khách sạn này": 'Bankcard',
        "Thời gian yên lặng": None,
        "Tiệc tùng": 'Cho phép tổ chức',
        "Trẻ em và giường": 'Trẻ em dưới 5 tuổi',
        "Vật nuôi": 'Cho phép mang vật nuôi',
        "Đặt cọc đề phòng hư hại có thể hoàn lại": 'Yêu cầu VND 1.000.000 tiền đặt cọc đề phòng hư hại khi đến nghỉ.',
        'room_service_included': 'Không cần thanh toán trước - thanh toán tại chỗ nghỉ\nKhông cần thẻ tín dụng'
    },

    # ====== room ======
    'capacity': 2,
    'room_type': 'Family',
    'room_level': 'Standard', 
    'area': 50,
    'bed_type': {'type': 'giường đôi', 'count': 1},
    'included_breakfast': True,
    'room_amenties':[],
    'room_facilities':[],
    'room_view': [],

}


In [5]:
ids_result = []
for id in feature_url[0:1]:
    ids_result.append(id['id'])


In [36]:
print( f"🔵 Bắt đầu tìm kiếm các khách sạn được đánh giá cao về dịch vụ của bạn.") 
if query['services']:
    print(f"✎ Dịch vụ: [bold #80CFFF]{', '.join(query['services'])}[/bold #80CFFF].")
    services = set()
    for criteria in query['services']:
        services.update(utils.convert_group(criteria, "services", "expand"))
    result_services = fgr.get_score_sub_rate(services, utils.filter_matching_elements(ids_result, feature_sub_rate))
    score_sub_rate_services = {item['id']: item['score'] for item in result_services}

🔵 Bắt đầu tìm kiếm các khách sạn được đánh giá cao về dịch vụ của bạn.

✎ Dịch vụ: Dịch vụ sự kiện & hỗ trợ đặc biệt.

In [ ]:
import numpy as np
import json

import sys
from pathlib import Path


import config


try:
    with open(config.get_path("feature_sub_rate.json"), 'r', encoding='utf-8') as f:
        feature_sub_rate = json.load(f)
except FileNotFoundError:
    print("File feature_sub_rate.json not found. Please check the file path.")
    feature_sub_rate = []

try:
    with open(config.get_path("weights_criteria_utilities.json"), 'r', encoding='utf-8') as f:
        weights_criteria_utilities = json.load(f)
except FileNotFoundError:
    print("File weights_criteria_utilities.json not found. Please check the file path.")
    weights_criteria_utilities = {}



def get_dynamic_weights(selected_utilities, utility_scores=weights_criteria_utilities):
    """
    Tạo trọng số động dựa trên các tiện ích mà người dùng chọn.

    Parameters:
    - selected_utilities (list of str): Danh sách các tiện ích người dùng quan tâm.
    - utility_scores (dict): Bảng trọng số giữa tiện ích và hạng mục đánh giá (mặc định lấy từ file JSON).

    Returns:
    - dict: Trọng số trung bình của các hạng mục đánh giá liên quan đến tiện ích đã chọn.
    """
    category_weights = {}
    
    for utility in selected_utilities:
        if utility in utility_scores:
            for category, weight in utility_scores[utility].items():
                if category in category_weights:
                    category_weights[category].append(weight)
                else:
                    category_weights[category] = [weight]
    
    return {category: np.mean(weights) for category, weights in category_weights.items()}


def normalize_hotel_scores(hotel_data):
    """
    Chuẩn hóa điểm đánh giá của từng khách sạn về khoảng [0, 1].

    Parameters:
    - hotel_data (dict): Dữ liệu điểm đánh giá từng hạng mục của một khách sạn.

    Returns:
    - dict: Dữ liệu đã được chuẩn hóa về khoảng 0-1 cho các trường số.
    """
    normalized_data = {}
    for key, value in hotel_data.items():
        try:
            normalized_data[key] = float(value.replace(",", ".")) / 10
        except (ValueError, AttributeError):
            normalized_data[key] = value / 10
    return normalized_data


def compute_hotel_score(hotel_data, selected_utilities):
    """
    Tính điểm tổng hợp cho khách sạn dựa trên trọng số động từ tiện ích được chọn.

    Parameters:
    - hotel_data (dict): Dữ liệu thô về điểm số từng tiêu chí của khách sạn.
    - selected_utilities (list of str): Danh sách tiện ích người dùng quan tâm.

    Returns:
    - float: Điểm tổng hợp của khách sạn.
    """
    category_weights = get_dynamic_weights(selected_utilities)
    normalized_data = normalize_hotel_scores(hotel_data)
    
    total_weight = sum(category_weights.values())
    final_score = 0
    for k in normalized_data.keys():
        if k in category_weights.keys():
            final_score += normalized_data[k] * (category_weights[k])
    return final_score / total_weight


def get_score_sub_rate(query: list[str], feature_sub_rate=feature_sub_rate):
    """
    Tính điểm và xếp hạng khách sạn dựa trên truy vấn tiện ích người dùng.

    Parameters:
    - query (list of str): Danh sách tiện ích người dùng muốn ưu tiên.
    - feature_sub_rate (list of dict): Danh sách thông tin khách sạn kèm theo điểm các hạng mục.

    Returns:
    - list of dict: Danh sách khách sạn gồm 'id' và 'score', được sắp xếp giảm dần theo điểm.
    """
    result = []
    for data in feature_sub_rate:
        result.append({'id': data['id'], 'score': compute_hotel_score(data, query)})
    
    return sorted(result, key=lambda x: x['score'], reverse=True)


In [7]:
print( f"🔵 Bắt đầu tìm kiếm các khách sạn được đánh giá cao về dịch vụ của bạn.") 
if query['services']:
    print(f"✎ Dịch vụ: [bold #80CFFF]{', '.join(query['services'])}[/bold #80CFFF].")
    services = set()
    for criteria in query['services']:
        services.update(utils.convert_group(criteria, "services", "expand"))
    result_services = get_score_sub_rate(services, utils.filter_matching_elements(ids_result, feature_sub_rate))

🔵 Bắt đầu tìm kiếm các khách sạn được đánh giá cao về dịch vụ của bạn.

✎ Dịch vụ: Dịch vụ sự kiện & hỗ trợ đặc biệt.

🔵 Trọng số hạng mục đánh giá: {'Nhân viên phục vụ': 0.75, 'Tiện nghi': 0.75, 'Sạch sẽ': 0.6875, 'Thoải mái': 0.75,
'Đáng giá tiền': 0.875, 'Địa điểm': 0.75, 'WiFi miễn phí': 0.5625}
🔵 Dữ liệu khách sạn đã chuẩn hóa: {'id': 1000059.3, 'stars_rating': 0.3, 'rate': 0.9199999999999999, 'Nhân viên 
phục vụ': 0.97, 'Tiện nghi': 0.9, 'Sạch sẽ': 0.93, 'Thoải mái': 0.9400000000000001, 'Đáng giá tiền': 0.93, 'Địa 
điểm': 0.9199999999999999, 'WiFi miễn phí': 0.8800000000000001}

Nhân viên phục vụ

Tiện nghi

...

Thoải mái

Đáng giá tiền

Địa điểm

WiFi miễn phí

🔵 Tổng trọng số: 5.125, Điểm cuối cùng: 4.745625

In [8]:
result_services

[{'id': '10000593', 'score': 0.9259756097560976}]

In [8]:

def get_best_hotels(query): 
    """    
    from geopy.geocoders import Nominatim
    from geopy.distance import geodesic
    import numpy as np

    max_distance_km = 18

    if query['location'] and query['distance_to_city_center']:
        max_distance_km = query['distance_to_city_center']
        print( f"🔵 Thực hiện tìm kiếm khách sạn cách trung tâm thành phố {', '.join(query['location'])} {max_distance_km} km ")
        result_location = address_nearplaces_recommendation.find_hotels_near_location(places = query['location'], max_distance_km = max_distance_km + 2)
    else:
        result_location = data_list

    if query['nearby_places']:
        print( f"🔵 Thực hiện tìm kiếm khách sạn gần {', '.join(query['nearby_places'])}")
        result_location = address_nearplaces_recommendation.find_hotels_near_location(data_list = result_location, places = query['nearby_places'], max_distance_km = 20)

    ids_resulf = {item['id'] for item in result_location}  # Tạo tập hợp chứa các id từ A
    
    if query['public_transport_access']:
        print( f"🔵 Thực hiện lọc khách sạn gần {', '.join(query['public_transport_access'])}")
    result_location = filter_hotels_by_distance(filter_matching_elements(ids_resulf, feature_address_nearplaces), 10)
    ids_resulf = {item['id'] for item in result_location} 
    print( f"✅ Hoàn thành thu thập khách sạn thỏa vị trí")

    del geodesic
    del Nominatim"""

    # ============================================================
    print(f"🔵 Bắt đầu tìm kiếm khách sạn thoe địa điểm")
    if query.get('nearby_places') or query.get('province'):
        filter_location_hotels = rl.find_hotels_near_location(
            feature_location, query.get('nearby_places', []), query.get('province'), max_distance_km=20
        )
        result_location_hotels_score = rl.get_location_score(
            filter_location_hotels, query.get('nearby_places', []),
            query.get('province'), query.get('is_near_center', False)
        )
    else:
        result_location_hotels_score = feature_location
        for hotel in result_location_hotels_score:
            hotel['location_score'] = 0

    # lấy ra danh sách các id khách sạn
    ids_result = []
    for id in result_location_hotels_score:
        ids_result.append(id['id'])

    
    print( f"🔵 Đã thu được danh sách các ID hotel thỏa điều kiện địa lý.")


    # ============================================================
    feature_star = []
    for data in feature_star_rating:
        if data['id'] in ids_result:
            feature_star.append(data)

    # 2. Filter star rating
    if query.get('stars_rating') not in [None, '', []]:
        result_stars_hotels_score = fgsp.get_score_rating(query, feature_star)
    else:
        result_stars_hotels_score = []
        for hotel in feature_star_rating:
            result_stars_hotels_score.append({
                'id': hotel['id'],
                'score_rating': 1
            })
    # OUTPUT: [{'id': '10000593', 'score_rating': 0.5}, {'id': '10000594', 'score_rating': 0.8}, ...]
    # loại bỏ các khách sạn có điểm đánh giá < 0.7. (chưa lọc)

    # ============================================================

    print( f"🔵 Xem như đã thu được danh sách các ID hotel_room thỏa điều kiện loại phòng và số lượng.")

    # mapping
    detail_room = []
    for detail_room_ in feature_detail_room:
        if detail_room_['id'] in ids_result:
            detail_room.append(detail_room_)

    # 3. Filter theo giá
    price = query.get('price')
    if price not in [None, '', [], (None, None)]:
        result_room_price_score = fgsp.get_price_score(query, detail_room)
        score_price = fgsp.get_max_room_score_per_hotel(result_room_price_score)
    else: 
        score_price = []
        for price in detail_room:
            score_price.append({
                'id': price['id'], 
                'hotel_score_price' : 1
            })
        # OUTPUT [{'id': '10000593', 'hotel_score_price': 0.9847}, ...]
        # kieẻm tra giá thực tế so với giá yêu cầu,  nhỏ hơn thì vẫn lấy lớn hơn thì căn nhăc 
    
    score_room = rr.get_room_info_score(query, detail_room)
    # OUTPUT : {'id': '10000593', 'room_id': 'RD1000059301', 'room_score': 0.6}
    

    # ============================================================
    print( f" Kiểm tra chính sách phù hợp.")
    filtered_policies = {} 
    for id in ids_result:
        if id in feature_policies:
            filtered_policies[id] = feature_policies[id]

    score_policies = {} 
    if query.get('policies'):
        for id, data in filtered_policies.items():
            score_policies[id] = rp.find_similar_hotel_policies(query['policies'], data)
            # OUTPUT: {'id':'score', ...}
    
    print( f" Đã tìm kiếm các chính sách phù hợp với yêu cầu của bạn.")

    # =============================================================
    print( f"🔵 Bắt đầu tìm kiếm các khách sạn được đánh giá cao về dịch vụ của bạn.") 
    if query['services']:
        print(f"✎ Dịch vụ: [bold #80CFFF]{', '.join(query['services'])}[/bold #80CFFF].")
        services = set()
        for criteria in query['services']:
            services.update(utils.convert_group(criteria, "services", "expand"))
        result_services = fgr.get_score_sub_rate(services, utils.filter_matching_elements(ids_result, feature_sub_rate))
        score_sub_rate_services = {item['id']: item['score'] for item in result_services}
        # OUTPUT: {'id':'score', ...}

    else:
        print(f"✎ Dịch vụ: Rỗng")
        print(f"✎ Dựa vào các tiện ích mà bạn yêu cầu: [bold #80CFFF]{', '.join(query['amenities'])}[/bold #80CFFF].")
        designated_utility = set()
        for amenity in query['amenities']:
            cluster = fga.find_clusters([amenity], threshold=0.01)
            designated_utility.update(cluster)
        designated_utility = list(designated_utility)
        print(f"✎ Nên sẽ ưu tiên các khách sạn được đánh cao với các tiêu chí đã phục thuộc các tiện ích trên.")
        score_sub_rate_services = fgr.get_score_sub_rate(designated_utility, 
                                                  utils.filter_matching_elements(ids_result, feature_sub_rate))
        # OUTPUT: {'id':'score', ...}
    
    print(" Đã tìm kiếm các khách sạn được đánh giá cao về dịch vụ của bạn.")


    # ============================================================
    if query['services']:
        print(f"🔵 Bắt đầu tìm kiếm các khách sạn có dịch vụ: [bold #80CFFF]{', '.join(query['services'])}[/bold #80CFFF]")
        services = set()
        for criteria in query['services']:
            services.update(utils.convert_group(criteria, "services", "expand"))
        score_services = fga.get_score_services(user_input = list(services), 
                                    List_ids = ids_result, 
                                    weights=[0.5, 0.5])
        score_services = {item['id']: item['final_score'] for item in score_services}
        # OUTPUT: {'id':'score', ...} score này chưa chuẩn hóa 
        
        print(f" Đã tìm kiếm các khách sạn khớp dịch vụ bạn quan tâm.")

    # ============================================================
    if query['amenities']:
        print(f"🔵 Bắt đầu tìm kiếm khách sạn chứa các tiện nghi [bold #80CFFF]{', '.join(query['amenities'])}[/bold #80CFFF]")
        result_facilities_hotel = recommend_faclities_hotel.predict_assignID(
            input_amenities= query['amenities'],
            hotel_ids=ids_result,
            similarity_threshold=0.9, 
            normalization_factor_base=10
        )
        result_facilities_room = recommend_faclities_room.predict_assignID(
            input_amenities= query['amenities'],
            hotel_ids=ids_result,
            similarity_threshold=0.9, 
            normalization_factor_base=10
        )

        score_facilities = fga.calculate_hotel_scores(result_facilities_hotel, 
                                   result_facilities_room, 
                                   threshold=0.9, 
                                   weight=[0.5, 0.5])
        
        print(f" Đã tìm kiếm các khách sạn có tiện nghi bạn quan tâm.")

    print(f"Danh sách khách sạn đã tìm kiếm được:")
    result_final_info_score = utils.compute_total_score(score_facilities, 
                                                        score_sub_rate_services, 
                                                        score_services,
                                                        score_policies,
                                                        weights=(0.3, 0.3, 0.3, 0.1))
    result_final_info_score = dict(result_final_info_score)
    # OUTPUT : {'id':'score', ...}


    # ============================================================
    print("============================================================")
    print("Giai đoạn 2:")
    print("Kiểm tra phản hòi của khách hàng")
    score_review_quality = hotel_review_stats.calculate_final_score_local_data(ids_result, print_warning=False)
    if not query['country'] and not query['state']: # TH không có thông tin về quốc gia và trạng thái
        if not query['criteria']: # TH không có thông tin về tiêu chí
            score_review = rhr.calculate_weighted_bayesian_score(
                result_groupby_id = {id: result_groupby_id[id] for id in ids_result if id in result_groupby_id},
                #user_weights = rhr.allocate_weights_with_ratios(), 
                C=100
                )
            score_review = dict(zip(score_review['hotel_id'], score_review['bayesian_score']))

            # Tính điểm cuối cùng
            result_final_review_score = rhr.calculate_final_score_from_reviews(
                score_review, 
                score_review_quality, 
                w=0.8, 
                q_default=0.5, 
                s_global=0.7, 
                threshold=0.3
            )
            result_final_review_score = dict(zip(result_final_review_score['id'], result_final_review_score['final_score']))
            # OUTPUT : {'id':'score', ...}
        else: # TH có thông tin về tiêu chí
            print(f"🔵 Tập trung mạnh vào khách sạn có tiêu chí: [bold #80CFFF]{', '.join(query['criteria'])}[/bold #80CFFF]")
            criterion = set()
            for criteria in query['criteria']:
                criterion.update(utils.convert_group(criteria, "criteria", "expand"))
            criterion = {key: 1 for key in criterion}
            weights = rhr.allocate_weights_with_ratios(criterion)
            score_review = rhr.calculate_weighted_bayesian_score(
                result_groupby_id = {id: result_groupby_id[id] for id in ids_result if id in result_groupby_id},
                user_weights = weights, 
                C=100
                )
            score_review = dict(zip(score_review['hotel_id'], score_review['bayesian_score']))
            result_final_review_score = rhr.calculate_final_score_from_reviews(
                score_review, 
                score_review_quality, 
                w=0.8, 
                q_default=0.5, 
                s_global=0.7, 
                threshold=0.2
            )
            result_final_review_score = dict(zip(result_final_review_score['id'], result_final_review_score['final_score']))
            # OUTPUT : {'id':'score', ...}

    else: # TH có thông tin về quốc gia và trạng thái
        print(f"🔵 Cảm ơn bạn đã cung cấp thông tin về quốc gia: {query['country']} và trạng thái: {query['state']}")
        result = CF.aggregate_prediction(hotel_ids = ids_result, 
                                            country = query['country'], 
                                            status = query['state'], 
                                            state_model=state_model, 
                                            country_model=country_model, 
                                            print_result=False, 
                                            print_warning=False,
                                            )
        if query['criteria']: # TH có thông tin về tiêu chí
            print(f"🔵 Tập trung mạnh vào khách sạn có tiêu chí: [bold #80CFFF]{', '.join(query['criteria'])}[/bold #80CFFF]")
            criterion = set()
            for criteria in query['criteria']:
                criterion.update(utils.convert_group(criteria, "criteria", "expand"))
            criterion = {key: 1 for key in criterion}
            weights = rhr.allocate_weights_with_ratios(criterion)
        else:
            weights = rhr.allocate_weights_with_ratios()

        score_review = {}
        for key, value in result.items():
            score_review[key] = sum(value[key1] * weights[key1] for key1 in value) / sum(weights.values())
        
        result_final_review_score = rhr.calculate_final_score_from_reviews(
            score_review, 
            score_review_quality, 
            w=0.8, 
            q_default=0.5, 
            s_global=0.7, 
            threshold=0.2
        )
        result_final_review_score = dict(zip(result_final_review_score['id'], result_final_review_score['final_score']))
        # OUTPUT : {'id':'score', ...}


    # Kết hợp các điểm số
    result_final_score = utils.compute_total_score(
        result_final_info_score, 
        result_final_review_score, 
        weights=(0.7, 0.3)
    )
        

    return result_final_score

In [9]:
get_best_hotels(query)

🔵 Bắt đầu tìm kiếm khách sạn thoe địa điểm

🔵 Đã thu được danh sách các ID hotel thỏa điều kiện địa lý.

🔵 Xem như đã thu được danh sách các ID hotel_room thỏa điều kiện loại phòng và số lượng.

Kiểm tra chính sách phù hợp.

Đã tìm kiếm các chính sách phù hợp với yêu cầu của bạn.

...

Giai đoạn 2:

Kiểm tra phản hòi của khách hàng

🔵 Cảm ơn bạn đã cung cấp thông tin về quốc gia: Việt Nam và trạng thái: Cặp đôi

🔵 Tập trung mạnh vào khách sạn có tiêu chí: Vị trí & môi trường

[('4464775', 0.5354375230431713),
 ('3563537', 0.4442356053324338),
 ('2956757', 0.4334844944477081),
 ('6921137', 0.43210306904939877),
 ('4824897', 0.43202227132250176),
 ('2028376', 0.4115760876873565),
 ('6536748', 0.41047774126212344),
 ('5173273', 0.4093007479626751),
 ('3147128', 0.4022864566763647),
 ('4852870', 0.39707279918399274),
 ('3919497', 0.39273467721874117),
 ('2125472', 0.38932973522976827),
 ('4946994', 0.3861880995604653),
 ('3133832', 0.3786680902881274),
 ('3384511', 0.3747964093663718),
 ('8803524', 0.373585602929558),
 ('3249623', 0.3721161115500971),
 ('331758', 0.363682682264757),
 ('2967978', 0.36361446522073915),
 ('13191337', 0.36340379135706363),
 ('4822969', 0.3628400072934315),
 ('2812915', 0.3614096718160698),
 ('5937836', 0.35717180093625595),
 ('5051247', 0.35682178884744636),
 ('6120309', 0.3564530233462774),
 ('6588380', 0.3553599119854238),
 ('2648923', 0.35340445306897156),
 ('262297', 0.35224991873064854),
 ('4181231', 0.3516486028944006),
 ('54